## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [125]:
from pathlib import Path
import os
from dotenv import load_dotenv

# 1. Start from the current working directory
cwd = Path.cwd()

# 2. Search for the .env file in the LCEL folder specifically
# If you are already IN the 'Building_Chatbot...' folder, the path is just ".env"
env_path = cwd / ".env"

# If it's not in the immediate folder, search one level up
if not env_path.exists():
    env_path = cwd.parent / ".env"

# Final check and load
if env_path.exists():
    load_dotenv(dotenv_path=env_path, override=True)
    print("✅ Loaded .env from:", env_path)
else:
    # Use glob to find the .env if you aren't sure where the kernel started
    possible_paths = list(cwd.glob("**/.env"))
    if possible_paths:
        env_path = possible_paths[0]
        load_dotenv(dotenv_path=env_path, override=True)
        print("✅ Found and loaded .env via search:", env_path)
    else:
        raise FileNotFoundError(f"Could not find .env anywhere in {cwd}")

print("`GROQ_API_KEY` present:", bool(os.getenv("GROQ_API_KEY")))

✅ Loaded .env from: D:\Projects\Gen\14. Langchain\Building_Chatbot_using_langchian\.env
`GROQ_API_KEY` present: True


In [126]:
import os
from langchain_groq import ChatGroq

# Ensure load_dotenv() was called before this
model = ChatGroq(
    model="llama-3.1-8b-instant",
    groq_api_key=os.getenv("GROQ_API_KEY")
)

In [127]:
import os

# 1. Get the string from the environment
my_key = os.getenv("GROQ_API_KEY")

# 2. Pass that string to the model
model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=my_key)

In [128]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Krish. As a Chief AI Engineer, I'm sure you have a fascinating job, overseeing the development and implementation of artificial intelligence solutions. What specific areas of AI are you most interested in or currently working on?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 48, 'total_tokens': 96, 'completion_time': 0.067750497, 'completion_tokens_details': None, 'prompt_time': 0.002243885, 'prompt_tokens_details': None, 'queue_time': 0.048983765, 'total_time': 0.069994382}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bef6d-a3f2-7b91-b010-b11625a036a0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 48, 'output_tokens': 48, 'total_tokens': 96})

In [129]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is Krish, and you are a Chief AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 117, 'total_tokens': 131, 'completion_time': 0.015314901, 'completion_tokens_details': None, 'prompt_time': 0.007407818, 'prompt_tokens_details': None, 'queue_time': 0.049485262, 'total_time': 0.022722719}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bef6d-a648-7fd1-8c23-ad205bdb6dd4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 117, 'output_tokens': 14, 'total_tokens': 131})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [130]:
!pip install langchain_community


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [131]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [132]:
config={"configurable":{"session_id":"chat1"}}

In [133]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [134]:
response.content

"Nice to meet you, Krish. As a Chief AI Engineer, I'm sure you have a fascinating role. What specific areas of AI are you most interested in or working on? Are you focusing on natural language processing, computer vision, or perhaps reinforcement learning?"

In [135]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="Your name is Krish, and you're a Chief AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 115, 'total_tokens': 129, 'completion_time': 0.015405308, 'completion_tokens_details': None, 'prompt_time': 0.006710414, 'prompt_tokens_details': None, 'queue_time': 0.053442326, 'total_time': 0.022115722}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bef6d-b1d9-70d3-83c9-dc3b70865742-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 115, 'output_tokens': 14, 'total_tokens': 129})

In [136]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"I don't have any information about your name as our conversation has just started. I'm happy to chat with you, though. Do you want to introduce yourself?"

In [137]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, nice to meet you. I'll do my best to remember that, but please keep in mind that I'm a large language model, I don't have personal memories like humans do, so I won't be able to recall your name in a future conversation if you don't tell me again."

In [138]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I remember you told me earlier. Your name is John.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [139]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [140]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hello Krish, it's nice to meet you. Is there anything I can help you with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 57, 'total_tokens': 78, 'completion_time': 0.033661083, 'completion_tokens_details': None, 'prompt_time': 0.00332017, 'prompt_tokens_details': None, 'queue_time': 0.05461855, 'total_time': 0.036981253}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bef6d-b566-7853-956c-5ba188252018-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 21, 'total_tokens': 78})

In [141]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [142]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hello Krish, it's nice to meet you. Is there anything I can help you with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 57, 'total_tokens': 78, 'completion_time': 0.034965251, 'completion_tokens_details': None, 'prompt_time': 0.002807433, 'prompt_tokens_details': None, 'queue_time': 0.052967847, 'total_time': 0.037772684}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bef6d-b63a-7f30-8a43-df7b2185a684-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 21, 'total_tokens': 78})

In [143]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.'

In [144]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [145]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष, मैं आपकी सहायता करने के लिए तैयार हूँ। आपके प्रश्नों का उत्तर देने के लिए मैं यहाँ हूँ। क्या आपके पास कोई सवाल या समस्या है?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [146]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [147]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष, मैं आपकी मदद करने के लिए यहाँ हूँ। आप किस बारे में पूछना चाहते हैं?'

In [148]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [149]:
response.content

'आपका नाम कृष है।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [152]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=lambda msgs: sum(len(m.content.split()) for m in msgs),
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs

In [153]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You like vanilla ice cream!'

In [154]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me the math problem 2 + 2.'

In [155]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [156]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'your name is Bob.'

In [157]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked the math problem 2 + 2.'